In [27]:
## direct read the small test set and training set from the pickle file
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

pickle_file = 'train_data.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  orders_train = save['orders_train']
  weather_train = save['weather_train'] 
  traffic_train = save['traffic_train']
  train_data = save['train_data']
  test_data = save['test_data']
  del save  # hint to help gc free up memory
  print('train_data', train_data.shape)
  print('weather_train', weather_train.shape)
  print('traffic_train', traffic_train.shape)
  print('train_data', train_data.shape)
  print('test_data', test_data.shape)

('train_data', (152251, 21))
('weather_train', (2528, 5))
('traffic_train', (184264, 7))
('train_data', (152251, 21))
('test_data', (2838, 22))


In [11]:
import pandas as pd
from time import time
import os
import glob
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import  cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint

In [28]:
train_data = train_data[train_data.time_slot > 40]
x= train_data.drop(['gap', 'date'], axis = 1)
y =train_data.gap


In [19]:
from operator import itemgetter
def report(grid_scores, n_top=3):
    """Report top n_top parameters settings, default n_top=3.

    Args
    ----
    grid_scores -- output from grid or random search
    n_top -- how many to report, of top models

    Returns
    -------
    top_params -- [dict] top parameter settings found in
                  search
    """
    top_scores = sorted(grid_scores,
                        key=itemgetter(1),
                        reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print(("Mean validation score: "
               "{0:.3f} (std: {1:.3f})").format(
               score.mean_validation_score,
               np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

    return top_scores[0].parameters

In [17]:
###search for the best parameters
def run_randomsearch(X, y, clf, para_dist, cv=5,
                     n_iter_search=20):
    """Run a random search for best Decision Tree parameters.

    Args
    ----
    X -- features
    y -- targets (classes)
    cf -- scikit-learn Decision Tree
    param_dist -- [dict] list, distributions of parameters
                  to sample
    cv -- fold of cross-validation, default 5
    n_iter_search -- number of random parameter sets to try,
                     default 20.

    Returns
    -------
    top_params -- [dict] from report()
    """
    random_search = RandomizedSearchCV(clf,
                        param_distributions=param_dist,
                        n_iter=n_iter_search)

    start = time()
    random_search.fit(X, y)
    print(("\nRandomizedSearchCV took {:.2f} seconds "
           "for {:d} candidates parameter settings.").format((time() - start), n_iter_search))

    top_params = report(random_search.grid_scores_, 3)
    return  top_params

In [22]:
###very time consuming step 
param_dist = {"n_estimators": randint(100, 1000),
              "max_features": randint(3, 15),
              "min_samples_split": randint(1, 20)}

regressor = RandomForestRegressor()
ts_rs = run_randomsearch(x, y, regressor, param_dist, cv=10, n_iter_search=50)


RandomizedSearchCV took 51240.56 seconds for 50 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.981 (std: 0.002)
Parameters: {'max_features': 14, 'min_samples_split': 9, 'n_estimators': 850}

Model with rank: 2
Mean validation score: 0.981 (std: 0.002)
Parameters: {'max_features': 14, 'min_samples_split': 9, 'n_estimators': 913}

Model with rank: 3
Mean validation score: 0.980 (std: 0.002)
Parameters: {'max_features': 14, 'min_samples_split': 12, 'n_estimators': 325}



In [29]:
##random forest 
regressor = RandomForestRegressor(n_estimators=850, max_features =14,  min_samples_split=9)
##regressor.fit(x, y)


In [36]:
from sklearn.cross_validation import  cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV

##custom scoring function 
def mape(estimator, X, y):
    y_pred = estimator.predict(X)
    df = pd.DataFrame()
    df['y'] = y
    df['y_pred'] = y_pred
    df.y_pred[df.y_pred == 0] = 1  ##need set the value to at least 1 
    df = df.round()
    df = df[df.y != 0]
    return ((np.sum(np.abs(df.y-df.y_pred)/df.y)/df.shape[0]))
    
scores = cross_val_score(regressor,x, y, cv=10, scoring=mape)
print("mean: {:.3f} (std: {:.3f})".format(scores.mean(), scores.std()))

/Library/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


mean: 0.556 (std: 0.275)


In [ ]:
##output predict values
# prepare prediction
x = test_data[['time_slot', 'start_district_id','demand','supply', 'dayType', 'Weather','Temperature','PM25','lv1','lv2','lv3','lv4','t-1','t-2','t-3','lv1_pect','lv2_pect','lv3_pect','lv4_pect']]
result = regressor.predict(x)
result =result.round()
result[result == 0] = 1 

final_result = pd.DataFrame(test_data)
final_result['gap'] = result
final_result.sort(['date', 'time_slot', 'start_district_id'], inplace= True)

In [ ]:
combinedTimeDistrict = lambda x: (x.date.strftime('%Y-%m-%d'))+'-'+str(x.time_slot)
    
final_result['Time_district'] = final_result.apply(combinedTimeDistrict, axis=1)
final_result.to_csv('result.csv',columns=['start_district_id', 'Time_district', 'gap'], header = False, index =False)

In [ ]:
final_result